# 1C. Data Cleaning: Zipcode Clusterings
<hr>

We take the same approach to cleaning the data as in our original version. We take zipcodes and cluster them into neighborhoods. This is better than using the original neighborhoods column as there were many missing values and non-uniform neighborhood names. By converting by cleaned zipcode, we can ensure that the clustering by neighborhoods is more accurate.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Masters Project/datasets/raw_datasets/')

In [0]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import math
%matplotlib inline

In [0]:
cols = [
    'id',
    'host_id',
    'zipcode',
    'property_type',
    'room_type',
    'accommodates',
    'bedrooms',
    'beds',
    'bed_type',
    'price',
    'number_of_reviews',
    'review_scores_rating',
    'host_listings_count',
    'availability_30',
    'minimum_nights',
    'bathrooms'
]

data = pd.read_csv(r'/content/drive/My Drive/Masters Project/datasets/raw_datasets/listings.csv', usecols=cols)

In [0]:
data.head(5)

,id,host_id,host_listings_count,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,minimum_nights,availability_30,number_of_reviews,review_scores_rating
0,958,1169,1.0,94117.0,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,$170.00,1,1,199,97.0
1,3850,4921,2.0,94131.0,House,Private room,2,1.0,1.0,1.0,Real Bed,$99.00,1,17,136,94.0
2,5858,8904,2.0,94110.0,Apartment,Entire home/apt,5,1.0,2.0,3.0,Real Bed,$235.00,30,0,111,98.0
3,7918,21994,10.0,94117.0,Apartment,Private room,2,4.0,1.0,1.0,Real Bed,$65.00,32,5,17,85.0
4,8142,21994,10.0,94117.0,Apartment,Private room,2,4.0,1.0,1.0,Real Bed,$65.00,32,30,8,93.0


In [0]:
data['zipcode'] = data['zipcode'].astype(str).replace('\.0', '', regex=True)

In [0]:
data.head()

,id,host_id,host_listings_count,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,minimum_nights,availability_30,number_of_reviews,review_scores_rating
0,958,1169,1.0,94117,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,$170.00,1,1,199,97.0
1,3850,4921,2.0,94131,House,Private room,2,1.0,1.0,1.0,Real Bed,$99.00,1,17,136,94.0
2,5858,8904,2.0,94110,Apartment,Entire home/apt,5,1.0,2.0,3.0,Real Bed,$235.00,30,0,111,98.0
3,7918,21994,10.0,94117,Apartment,Private room,2,4.0,1.0,1.0,Real Bed,$65.00,32,5,17,85.0
4,8142,21994,10.0,94117,Apartment,Private room,2,4.0,1.0,1.0,Real Bed,$65.00,32,30,8,93.0


In [0]:
# remove NaN values from dataframe
data = data.dropna(how='any', subset=['zipcode', 'property_type', 'bedrooms', 'beds', 'bathrooms'])

In [0]:
data['zipcode'].isnull().sum()

0

In [0]:
data.zipcode.unique()

array(['94117', '94131', '94110', '94115', '94107', '94109', '94102',
       '94114', '94105', '94133', '94121', '94118', '94127', '94103',
       '94104', '94122', '94123', '94112', '94108', '94124', '94111',
       '94113', '94116', '94129', '94132', 'nan', '94134', '94158',
       '94014', '94015', '94080'], dtype=object)

In [0]:

# convert formatting for price
data['price'] = (data['price'].str.replace(r'[^-+\d.]', '').astype(float))

# drop any inconsistent values
data = data[data['accommodates'] != 0]
data = data[data['bedrooms'] != 0]
data = data[data['beds'] != 0]
data = data[data['price'] != 0.00]

In [0]:
# turn NaN scores with 0 reviews into 'No Reviews'
idx_vals = data['review_scores_rating'][data['number_of_reviews'] == 0].index.values.tolist()
data.loc[idx_vals, ('review_scores_rating')] = data['review_scores_rating'][data['number_of_reviews'] == 0].replace('NaN', 'No Reviews')

# remove inconsistent NaN values
data = data[~data['review_scores_rating'].isnull()]

In [0]:
# convert review_scores_rating into buckets
def convert_scores_buckets(val):
    if val == 'No Reviews':
        return 'No Reviews'
    elif val >= 95.0:
        return '95-100'
    elif val >= 90.0 and val < 95.0:
        return '90-94'
    elif val >= 85.0 and val < 90.0:
        return '85-89'
    elif val >= 80.0 and val < 85.0:
        return '80-84'
    elif val >= 70.0 and val < 80.0:
        return '70-79'
    elif val >= 60.0 and val < 70.0:
        return '60-69'
    elif val >= 50.0 and val < 60.0:
        return '50-59'
    elif val >= 40.0 and val < 50.0:
        return '40-49'
    elif val >= 30.0 and val < 40.0:
        return '30-39'
    elif val >= 20.0 and val < 30.0:
        return '20-29'
    elif val >= 10.0 and val < 20.0:
        return '10-19'
    elif val < 10.0:
        return '0-9'
    
data['review_scores_rating'] = data['review_scores_rating'].apply(convert_scores_buckets)

In [0]:
data.head()

,id,host_id,host_listings_count,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,minimum_nights,availability_30,number_of_reviews,review_scores_rating
0,958,1169,1.0,94117,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,170.0,1,1,199,95-100
1,3850,4921,2.0,94131,House,Private room,2,1.0,1.0,1.0,Real Bed,99.0,1,17,136,90-94
2,5858,8904,2.0,94110,Apartment,Entire home/apt,5,1.0,2.0,3.0,Real Bed,235.0,30,0,111,95-100
3,7918,21994,10.0,94117,Apartment,Private room,2,4.0,1.0,1.0,Real Bed,65.0,32,5,17,85-89
4,8142,21994,10.0,94117,Apartment,Private room,2,4.0,1.0,1.0,Real Bed,65.0,32,30,8,90-94


We now will cluster the zipcodes together based on their neighborhood as described at the following link: http://www.healthysf.org/bdi/outcomes/94117.htm

We also encoded zip codes not listed here by placing them with their neighbors. We do this clustering because the neighborhood might have more important information than a zipcode. People would be more likely to search a specific neighborhood on air BN

In [0]:
# convert zipcodes into neighbrohoods

dict = {
    '94014' : 'Park Merced Twin Peaks',
    '94015' : 'Park Merced Twin Peaks',
    '94080' : 'South San Francisco',
    '94102' : 'Bay View Hunters Point',
    '94103' : 'Soma Tenderloin',
    '94104' : 'Chinatown',
    '94105' : 'Chinatown',
    '94107' : 'Soma Tenderloin',
    '94108' : 'Chinatown',
    '94109' : 'Cow Hollow Marina Pacific Heights',
    '94110' : 'Mission Bernal Heights',
    '94111' : 'Chinatown',
    '94112' : 'Castro Glen Park Noe Valley',
    '94113' : 'Castro Glen Park Noe Valley',
    '94114' : 'Castro Glen Park Noe Valley',
    '94115' : 'Haight Hayes Valley',
    '94116' : 'Sunset',
    '94117' : 'Haight Hayes Valley',
    '94118' : 'Cow Hollow Marina Pacific Heights',
    '94121' : 'Richmond',
    '94122' : 'Sunset',
    '94123' : 'Cow Hollow Marina Pacific Heights',
    '94124' : 'Bay View Hunters Point',
    '94127' : 'Park Merced Twin Peaks',
    '94129' : 'Cow Hollow Marina Pacific Heights',
    '94131' : 'Park Merced Twin Peaks',
    '94132' : 'Sunset',
    '94133' : 'Chinatown',
    '94134' : 'Mission Bernal Heights',
    '94158' : 'Soma Tenderloin',
    'nan'   : 'Not Listed'   
    }


def convert_zips_hoods(val):
     return dict[str(val)]

data['neighborhood'] = data['zipcode'].apply(convert_zips_hoods) 

In [0]:
data = data.drop('zipcode', 1)

In [0]:
data.head()

,id,host_id,host_listings_count,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,minimum_nights,availability_30,number_of_reviews,review_scores_rating,neighborhood
0,958,1169,1.0,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,170.0,1,1,199,95-100,Haight Hayes Valley
1,3850,4921,2.0,House,Private room,2,1.0,1.0,1.0,Real Bed,99.0,1,17,136,90-94,Park Merced Twin Peaks
2,5858,8904,2.0,Apartment,Entire home/apt,5,1.0,2.0,3.0,Real Bed,235.0,30,0,111,95-100,Mission Bernal Heights
3,7918,21994,10.0,Apartment,Private room,2,4.0,1.0,1.0,Real Bed,65.0,32,5,17,85-89,Haight Hayes Valley
4,8142,21994,10.0,Apartment,Private room,2,4.0,1.0,1.0,Real Bed,65.0,32,30,8,90-94,Haight Hayes Valley


In [0]:
data.to_csv(r'/content/drive/My Drive/Masters Project/datasets/clean_datasets/neighborhood_clustered_clean.csv', index=False)

In [0]:
# encode categorical variables
neighborhood_dummies = pd.get_dummies(data['neighborhood'])
property_dummies = pd.get_dummies(data['property_type'])
room_dummies = pd.get_dummies(data['room_type'])
bed_dummies = pd.get_dummies(data['bed_type'])
ratings_scores_dummies = pd.get_dummies(data['review_scores_rating'])

# replace the old columns with our new one-hot encoded ones
df = pd.concat((data.drop(['neighborhood', \
    'property_type', 'room_type', 'bed_type', 'review_scores_rating'], axis=1), \
    neighborhood_dummies.astype(str), property_dummies.astype(int), \
    room_dummies.astype(int), bed_dummies.astype(int), ratings_scores_dummies.astype(int)), \
    axis=1)

# move target predictor 'price' to the end of the dataframe
cols = list(df.columns.values)
idx = cols.index('price')
rearrange_cols = cols[:idx] + cols[idx+1:] + [cols[idx]]
df = df[rearrange_cols]

In [0]:
# convert non-categorical variables to floats and standardize
def standardize_col(col):
    mean = np.mean(col)
    std = np.std(col)
    return col.apply(lambda x: (x - mean) / std)

non_cat_vars = ['accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'host_listings_count', 'availability_30', 'minimum_nights', 'bathrooms']
for col in non_cat_vars:
    df[col] = df[col].astype(float)
    df[col] = standardize_col(df[col])
    
df.head()

,id,host_id,host_listings_count,accommodates,bathrooms,bedrooms,beds,minimum_nights,availability_30,number_of_reviews,Bay View Hunters Point,Castro Glen Park Noe Valley,Chinatown,Cow Hollow Marina Pacific Heights,Haight Hayes Valley,Mission Bernal Heights,Not Listed,Park Merced Twin Peaks,Richmond,Soma Tenderloin,South San Francisco,Sunset,Aparthotel,Apartment,Bed and breakfast,Boat,Boutique hotel,Bungalow,Cabin,Camper/RV,Condominium,Cottage,Dome house,Earth house,Guest suite,Guesthouse,Hostel,Hotel,House,Loft,Other,Resort,Serviced apartment,Timeshare,Tiny house,Townhouse,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,20-29,30-39,40-49,50-59,60-69,70-79,80-84,85-89,90-94,95-100,price
0,958,1169,-0.174825,-0.166102,-0.442291,-0.615425,0.115255,-0.013405,-0.705967,1.715323,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,170.0
1,3850,4921,-0.169405,-0.661477,-0.442291,-0.615425,-0.708373,-0.013405,1.115767,0.952290,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,99.0
2,5858,8904,-0.169405,0.824647,-0.442291,0.560571,0.938882,-0.013383,-0.819825,0.649499,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,235.0
3,7918,21994,-0.126043,-0.661477,2.843352,-0.615425,-0.708373,-0.013381,-0.250533,-0.488996,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,65.0
4,8142,21994,-0.126043,-0.661477,2.843352,-0.615425,-0.708373,-0.013381,2.595925,-0.598001,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,65.0


In [0]:
# log transform the response 'price'
df['price_log'] = df['price'].apply(lambda x: math.log(x))

In [0]:
# read to csv
df.to_csv(r'/content/drive/My Drive/Masters Project/datasets/clean_datasets/listings_neighborhood_clean.csv', index=False)